In [3]:
import yfinance as yf
ticker = "AMER3.SA"
acao = yf.download(ticker, start='2022-02-02', end='2023-02-02')

In [4]:
acao = yf.download(ticker, start='2022-02-02', end='2023-02-02')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-02,31.330000,32.439999,30.580000,31.209999,31.209999,7300100
2022-02-03,31.360001,32.369999,30.670000,32.000000,32.000000,6719900
2022-02-04,32.160000,32.209999,30.530001,32.150002,32.150002,7005100
2022-02-07,32.299999,32.759998,31.730000,32.200001,32.200001,3894500
2022-02-08,32.070000,33.529999,31.120001,33.270000,33.270000,6669900
...,...,...,...,...,...,...
2023-01-26,0.950000,1.050000,0.940000,1.030000,1.030000,86642800
2023-01-27,1.050000,1.200000,1.040000,1.200000,1.200000,98652200
2023-01-30,1.310000,1.640000,1.280000,1.450000,1.450000,204452600


In [15]:
df = yf.download(ticker, start='2022-02-02', end='2023-02-02').reset_index()

[*********************100%***********************]  1 of 1 completed


In [16]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

df = yf.download(ticker, start='2022-02-02', end='2023-02-02')

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['AAPL.Open'],
                high=df['AAPL.High'],
                low=df['AAPL.Low'],
                close=df['AAPL.Close'])])

fig.show()

[*********************100%***********************]  1 of 1 completed


KeyError: 'Date'

In [14]:
df.reset_index()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-02-02,31.330000,32.439999,30.580000,31.209999,31.209999,7300100
1,2022-02-03,31.360001,32.369999,30.670000,32.000000,32.000000,6719900
2,2022-02-04,32.160000,32.209999,30.530001,32.150002,32.150002,7005100
3,2022-02-07,32.299999,32.759998,31.730000,32.200001,32.200001,3894500
4,2022-02-08,32.070000,33.529999,31.120001,33.270000,33.270000,6669900
...,...,...,...,...,...,...,...
246,2023-01-26,0.950000,1.050000,0.940000,1.030000,1.030000,86642800
247,2023-01-27,1.050000,1.200000,1.040000,1.200000,1.200000,98652200
248,2023-01-30,1.310000,1.640000,1.280000,1.450000,1.450000,204452600
249,2023-01-31,1.610000,1.750000,1.580000,1.750000,1.750000,212962100


In [1]:
# Raw Package
import numpy as np
import pandas as pd

# Market Data 
import yfinance as yf

#Graphing/Visualization
import datetime as dt 
import plotly
import plotly.graph_objs as go 

# Override Yahoo Finance 
yf.pdr_override()

# Create input field for our desired stock 
stock=input("Enter a stock ticker symbol: ")

# Retrieve stock data frame (df) from yfinance API at an interval of 1m 
df = yf.download(tickers=stock,period='1d',interval='1m')

df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA20'] = df['Close'].rolling(window=20).mean()

# MACD
macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)

# stochastic
stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)


# Declare plotly figure (go)
fig=go.Figure()

# add subplot properties when initializing fig variable
fig = plotly.subplots.make_subplots(rows=4, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.5,0.1,0.2,0.2])

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = 'market data'))
    
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))

# Plot volume trace on 2nd row
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors
                    ), row=2, col=1)

# Plot MACD trace on 3rd row
colorsM = ['green' if val >= 0 
          else 'red' for val in macd.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     marker_color=colorsM
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)

# Plot stochastics trace on 4th row
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2)
                        ), row=4, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         line=dict(color='blue', width=1)
                        ), row=4, col=1)

# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False)
                  

# Make the title dynamic to reflect whichever stock we are analyzing
fig.update_layout(
    title= str(stock)+' Live Share Price:',
    yaxis_title='Stock Price (USD per Shares)') 

# update y-axis label
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
fig.update_yaxes(title_text="Stoch", row=4, col=1)           

fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)



fig.show()

[*********************100%***********************]  1 of 1 completed


NameError: name 'MACD' is not defined

In [6]:
import requests
import json
import numpy as np
import pandas as pd
# AlphaVantage Demo IBM API
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo'
response = requests.get(url)
# Load text to clean json object
json_load = json.loads(response.text)
# 'Time Series (Daily)' consists the time stamps for the demo
df = pd.DataFrame(json_load['Time Series (Daily)']).T
# Rename column names & change datetime to pandas datetime
df.reset_index(inplace=True)
df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
df['datetime'] = pd.to_datetime(df['datetime'])
display(df)

WINDOW = 30
df['sma'] = df['close'].rolling(WINDOW).mean()
df['std'] = df['close'].rolling(WINDOW).std(ddof = 0)
display(df)

,datetime,open,high,low,close,volume
0,2023-02-08,135.7100,136.7400,135.1600,135.9800,4593748
1,2023-02-07,135.6700,136.4000,134.4500,135.8400,3737553
2,2023-02-06,135.8300,136.3200,134.9500,136.1800,4841300
3,2023-02-03,136.3500,136.9500,135.5300,136.9400,3755720
4,2023-02-02,135.9600,136.7200,134.8500,136.3900,6107793
...,...,...,...,...,...,...
95,2022-09-22,124.7600,126.5100,124.4100,125.3100,4038335
96,2022-09-21,126.8900,127.8400,124.9200,124.9300,3389545
97,2022-09-20,126.9000,126.9900,125.5200,126.3000,2837485
98,2022-09-19,126.4900,128.0600,126.2800,127.7300,3981606


,datetime,open,high,low,close,volume,sma,std
0,2023-02-08,135.7100,136.7400,135.1600,135.9800,4593748,NaN,NaN
1,2023-02-07,135.6700,136.4000,134.4500,135.8400,3737553,NaN,NaN
2,2023-02-06,135.8300,136.3200,134.9500,136.1800,4841300,NaN,NaN
3,2023-02-03,136.3500,136.9500,135.5300,136.9400,3755720,NaN,NaN
4,2023-02-02,135.9600,136.7200,134.8500,136.3900,6107793,NaN,NaN
...,...,...,...,...,...,...,...,...
95,2022-09-22,124.7600,126.5100,124.4100,125.3100,4038335,125.870667,6.729720
96,2022-09-21,126.8900,127.8400,124.9200,124.9300,3389545,125.474000,6.415428
97,2022-09-20,126.9000,126.9900,125.5200,126.3000,2837485,125.077333,5.968647
98,2022-09-19,126.4900,128.0600,126.2800,127.7300,3981606,124.725333,5.469574


In [8]:
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = ('IBM', 'Volume'), vertical_spacing = 0.1, row_width = [0.2, 0.7])

# ----------------
# Candlestick Plot
fig.add_trace(go.Candlestick(x = df['datetime'],
                             open = df['open'],
                             high = df['high'],
                             low = df['low'],
                             close = df['close'], showlegend=False,
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
fig.add_trace(go.Scatter(x = df['datetime'],
                         y = df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
fig.add_trace(go.Scatter(x = df['datetime'],
                         y = df['sma'] + (df['std'] * 2),
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1)

# Lower Bound fill in between with parameter 'fill': 'tonexty'
fig.add_trace(go.Scatter(x = df['datetime'],
                         y = df['sma'] - (df['std'] * 2),
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)


# ----------------
# Volume Plot
fig.add_trace(go.Bar(x = df['datetime'], y = df['volume'], showlegend=False), 
              row = 2, col = 1)

# Remove range slider; (short time frame)
fig.update(layout_xaxis_rangeslider_visible=False)

# Stock data has gaps in dates, specifically in weekends and holidays
# create a list of dates that are NOT included from start to end
date_gaps = [date for date in pd.date_range(start = '2020-12-21', end = '2021-05-14') if date not in df['datetime'].values]

# Update Xaxes 
fig.update_xaxes(rangebreaks = [dict(values = date_gaps)])

fig.show()